# Research Notebook
## Felix Groh
## Date: 27.01.2025 - 17.02.2025

# 1: Experience
## Describe at least one research activity you worked on this week. 
- Implemented computation of coefficients $c^{(1)}$ and $c^{(2)}$ and by extension $A_\ell$ and $B_\ell$.
- Calculated the cross-sections for inelastic scattering and viscosity
- Implemented the convergence test after Ref. [1] section (5.3)


- Bug fixes and attempts at better code praxis 
    - Made minor changes to the implementation of the Schrödinger equation to only be dependent on dimensionless parameters $a, b, c, d$ where $c^2 = a^2 - d^2$

# 2: What? (What happened?)
## Solution to the two state Schrödinger equation and its resulting coefficients

From the now derived coefficients it is possible to calculate the different cross-sections: (For derivation of these coefficients see (6.2) in the appendix)
\begin{equation} \begin{pmatrix}
    C^{(1)} \\
    C^{(2)}
\end{pmatrix} = \begin{pmatrix}
h_l^{(1)}(kr_m) & h_l^{(2)}(kr_m) \\
h_l^{(1)'}(kr_m) & h_l^{(2)'}(kr_m)
\end{pmatrix}^{-1} \begin{pmatrix}
\chi(r_m) \\
\chi'(r_m)
\end{pmatrix}
\end{equation}

In Ref. [1] it is useful to define $k$ to be a matrix, since this allows us to compute the cross sections from eqs. (24), (25) and (26) in one step, instead of component wise, which helps decreasing algorithm runtime, because the Legendre-Polynomial $P_\ell (\cos( \theta))$ now only needs to be calculated once per integration step instead of four-times. Since all viable integration algorithms from libraries such as `numpy` or `scipy` are only capable of one dimensional integrands, not $2 \times 2$ matrices a transformation of the integral is necessary, in order to benefit from the possible time gains of calculating the cross-sections matrix wise.

To solve ODEs of first order, it is thus often useful to transform them in to integrals using a technique known as `quadrature`. (See (6.1) in the appendix)

## Newly formulated coefficients and formulae to calculate cross-sections 

In our case we only want specific cross-sections:
\begin{align}
    \sigma_{\rm in} &= \frac{\pi}{k_1 k_2} \sum_\ell (2\ell + 1) |B_{\ell}|^2, \\
    \sigma_{\rm vis} &= \frac{\pi}{k_1^2} \sum_{\ell} \frac{(\ell+2)(\ell+1)}{2\ell+3} \left| A_{\ell+2} - A_\ell \right|^2,
\end{align}
where
\begin{align}
    A_{\ell} &= -1 + \sum_{j=1,2} \left[\mathbf{c}_\ell^{(1)} \right]_{1j} \left[(\mathbf{c}_\ell^{(2)})^{-1} \right]_{j1} \\
    B_\ell &= \sum_{j=1,2} \left[\mathbf{c}_\ell^{(1)} \right]_{2j} \left[(\mathbf{c}_\ell^{(2)})^{-1} \right]_{j1} \, .
\end{align}

Since $c_\ell^{(1) \, ij}$ and $c_\ell^{(2) \, ij}$ are the primary contributions of $k \in \mathbb{R}^{2 \times 2}$ it is still useful to let $k$ be a matrix for purposes of calculating those coefficients. However, because we transform to dimensionless parameters, k is now defined as $k \coloneqq \sqrt{a^2 - \delta_{i2} d^2} x_m$. With that our cross-sections transform to:
\begin{align}
    \bar{\sigma}_{\rm vis} &= \frac{\pi}{a^2 b^2} \sum_{\ell} \frac{(\ell+2)(\ell+1)}{2\ell+3} \left| A_{\ell+2} - A_\ell \right|^2  \\
    \bar{\sigma}_{\rm in} &= \frac{\pi}{2acb^2} \sum_\ell (2\ell + 1) |B_\ell|^2
\end{align}

After that however, when $A_\ell$ and $B_\ell$ are calculated it is more useful to transform $k$ into an array where $k \coloneqq {\rm diag \,} (k)$ we only view the on-diagonal components.

Now a calculation of those cross-sections $\sigma_{\rm in}$ and $\sigma_{\rm vis}$ can easily be achieved by following the above formulae:
``` python
def compute_cross_sections(a, b, d, x_range, L_max, max_iter, convergenceRequirement = 0.01) -> NDArray:
    coefficients = compute_coefficients(a, b, d, x_range, L_max + 2, max_iter + 1, convergenceRequirement)  # L_max + 2 to accommodate the appearance of L + 2 in viscosity cross-section 
    A_l = coefficients[0,:max_iter]
    B_l = coefficients[1,:max_iter]
    
    inelastic_cross_section, viscosity_cross_section = 0, 0
    for L in range(L_max + 1):
        inelastic_cross_section += (2*L + 1) * abs(B_l[L])**2
        viscosity_cross_section += (L + 2) * (L + 1) / (2*L + 3) * abs(A_l[L + 2]- A_l[L])**2
    
    c = np.sqrt(a**2 - d**2)

    inelastic_cross_section *= np.pi / (a**2 * b**2)
    viscosity_cross_section *= np.pi / (2 * a * b**2 * c) 

    cross_sections = np.array([inelastic_cross_section, viscosity_cross_section])

    return cross_sections
```
Note, that in order to calculate the cross-section $\sigma_{\rm vis}$ for mode `L = 0` it is necessary to compute the coefficient $A_\ell$ up to $A_{\ell + 2}$.

## Convergence test, and computational approach

To ensure the interval for which the Schrödinger equation is solved is sufficiently wide, and the cross-sections are convergent for the $\ell-$ mode, it is necessary to implement a convergence test. This is done after Ref[1] Section (5.3). However, contrary to the algorithm developed there, we switch the order for the $x_i$ and $x_m$ convergence tests, since the solution for $\chi$ is heavily dependent on $x_i$ as the initial conditions are directly computed from it. 
\begin{align}
    \chi{(x_i)} &= \mathbb{I} \cdot x^{\ell + 1} \\
    \chi^{'}{(x_i)} &= \mathbb{I} \cdot (\ell + 1) x^\ell
\end{align}
Therefor the algorithm we used is as follows:
First decrease $x_i$ until convergence of $f_\ell$ or in our case $A_\ell$ and $B_\ell$ is achieved to within $1 \%$, then increase $x_m$ again until convergence to within $1 \%$ is achieved. Now increase $\ell = 0$ iteratively until until the convergence requirement of the cross-sections is met. To avoid a spurious convergence, the cross-sections then have to remain convergent for at least $10$ iterations. 

# 3: So what? (What does it mean?)
## Running the convergence test
Trying to run the `compute_cross_sections` function currently results in an exception, as a convergence for $x_m$ cannot be reached with in a reasonably set maximum iterations argument. This might be caused by a failure of the Schrödinger function it self to converge, by a flaw in the convergence test, or an error in the calculation of the asymptotic limit.
As the Schrödinger equation it self was checked by multiple people, an error there is unlikely. Since a very similar convergence test is used for establishing the convergence of $x_i$ an error there is also quit improbable. Therefor the most likely cause of the convergence test's failure is a problem with computation of the asymptotic behavior. Possible error there could include a flaw while calculating the hankel functions and their derivatives. 
What is quit extraordinary, is, that for increasing $x_m$ the coefficients $c^{(1)}$ and $c^{(2)}$ decrees with each iterations by exactly $29 \%$.


# 4. Now what? (What's next?)
## Determining root of convergence problem, and eliminating it
The primary target now is, to eliminate the convergence test's problem. After that is eliminated, it would be prudent to run tests that establish agreement between Debbie's analytic solution (See Ref. [2]) and the numeric solution for `L=0`.
Concurrently a transform back to dimensionful cross-sections should be implemented. 

When the numerical solution is working, and the SIM-team has implemented the hydrodynamical fluid simulation taking into account the 2-state nature of the problem, - meaning they have implemented upscatering and downscattering -, a linking of the respective programs will have to be managed.


# 5. Bibliography
[1] Durand, L. (n.d.). InSIDious matter (S. Tulin, Supervisor). Department of Physics and Astronomy, York University, Canada.

[2] Schutz, K., & Slatyer, T. R. (2015). Self-scattering for dark matter with an excited state. Journal of Cosmology and Astroparticle Physics, 2015(01), 021.

[3] Tulin, S., Yu, H. B., & Zurek, K. M. (2013). Beyond collisionless dark matter: particle physics dynamics for dark matter halo structure. Physical Review D—Particles, Fields, Gravitation, and Cosmology, 87(11), 115007.

[4] Blennow, M., Clementz, S., & Herrero-Garcia, J. (2017). Self-interacting inelastic dark matter: A viable solution to the small scale structure problems. Journal of Cosmology and Astroparticle Physics, 2017(03), 048.


# 6. Appendix

### 1. Computational optimization using reversed quadrature
`Quadrature` describes a process, where first order ODEs are transformed in to integrals, in order to solve them.
We can now reverse this approach and transform any integral of the form
\begin{equation}
    I = \int_a^b f(t) \, dt.
\end{equation}
Transforming the integral in to a first order ODE then looks like this:
\begin{equation}
    \frac{dF(t)}{dt} = f(t), \quad   F(a) = 0
\end{equation}

Applying this approach we can transform the integrals from (24-26), and then apply a standard ODE solver like `solve_ivp` or `ode_int`. Thus the Legendre-polynomials only have to be calculated once per infinitesimally small integration step $\Delta x$ instead of four times.
The Approach to calculate the cross-sections
\begin{align}
    \sigma^{ij} &= \int \frac{d \sigma}{dt} \sin (\theta) \, d \theta d \phi \\
    \sigma_{\rm T / rate}^{ij} &= \int \frac{d \sigma}{dt} (1 - \delta_{ij} \cos (\theta) ) \sin (\theta) \, d \theta d \phi \\
    \sigma_{\rm T}^{ij} &= \int \frac{d \sigma}{dt} (1 - \cos (\theta) ) \sin (\theta) \, d \theta d \phi \\
\end{align}

as described above, is to transform these integrals using a `reversed quadrature`. They then can be put into differential cross-section functions,
``` python 
def diff_pr_cross_section(theta, f0, k, f, L_max = 0) -> float:  
    diff_cross = k[0, 0] / k[1, 1] * abs(angular_dependence(theta, f, L_max))**2 * np.sin(theta)

    return diff_cross.flatten()
``` 
and calculated by using `solve_ivp`:
``` python
 def calculate_cross_sections(k, a, c, m_phi, aX, x_range, L_max, rtol=1e-12, atol=1e-12) -> NDArray: 
    f = np.zeros((L_max + 1, 2, 2), dtype = 'complex_')

    for L in range(L_max + 1):
        f[L] = scattering_amplitude(k, a, c, m_phi, aX, x_range, L) 

    init_cond = np.array([0, 0, 0, 0])

    purely_rate_cross_section = solve_ivp(diff_pr_cross_section, [0, np.pi], init_cond, rtol = rtol, atol = atol, args=(k, f, L_max, ))
```
Where the scattering amplitude and angular dependence are calculated using:
``` python
def scattering_amplitude(k, a, c, m_phi, aX, x_range, L) -> NDArray:  # Calculates f * A_l as this is the last step without theta dependence
    C = C_ij(k, a, c, m_phi, aX, x_range, L)
    C1 = C[:2][:2]
    C2 = C[2:][:2]

    f = np.dot(C1, inv(C2)) - np.identity(2)
    f *= 1j**L * (2*L + 1) / 2

    return f

def angular_dependence(theta, f, L_max = 0) -> NDArray:
    f_angular = 0
    for L in range(L_max + 1):
        f_angular += legendre(L)(np.cos(theta)) * f[L]
    return f_angular
```
Similarly this can be achieved for the other cross-sections. 

### 2. Determining the coefficients needed to compute the cross-sections
In order to determine the coefficients we match the numerical solution to the Schrödinger equation to the asymptotic wavefunction. At large $x \gg 0, x = x_m$ the solution converges, and the potential becomes ineffective:  
\begin{equation}
\chi(x) \sim h_l^{(1)}(kx) C^{(1)} + h_l^{(2)}(kx) C^{(2)},
\end{equation}
where $C^{(1)}$ and $C^{(2)}$ are $2 \times 2$ matrices, containing the coefficients for the two state solution. The wavefunction as well as its derivatives can be written as: 
\begin{align}
\chi(x_m) &= h_l^{(1)}(kx_m) C^{(1)} + h_l^{(2)}(kx_m) C^{(2)}, \\
\chi'(x_m) &= \frac{d}{dr}\left[h_l^{(1)}(kx) C^{(1)} + h_l^{(2)}(kx) C^{(2)}\right]_{x=x_m}
\end{align}
The derivatives of the Hankel functions are:
\begin{align}
h_l^{(1)'}(kx) &= k \left[\frac{l}{kx}h_{l}^{(1)}(kx) - h_{l + 1}^{(1)}(kx)\right], \\
h_l^{(2)'}(kx) &= k \left[\frac{l}{kx}h_{l}^{(2)}(kx) - h_{l + 1}^{(2)}(kx)\right]
\end{align}
For $r = r_m$ this gives us two equations:
\begin{align}
\chi(x_m) &= h_l^{(1)}(kx_m) C^{(1)} + h_l^{(2)}(kx_m) C^{(2)} \\
\chi'(x_m) &= h_l^{(1)'}(kx_m) C^{(1)} + h_l^{(2)'}(kx_m) C^{(2)}
\end{align}
These can be rewritten as a linear system.
\begin{equation}
\begin{pmatrix}
h_l^{(1)}(kx_m) & h_l^{(2)}(kx_m) \\
h_l^{(1)'}(kx_m) & h_l^{(2)'}(kx_m)
\end{pmatrix} \cdot \begin{pmatrix}
    C^{(1)} \\
    C^{(2)}
\end{pmatrix} = \begin{pmatrix}
\chi(x_m) \\
\chi'(x_m)
\end{pmatrix}
\end{equation}
Now we can solve for $c^{(1)}$ and $c^{(2)} $:
\begin{equation} \begin{pmatrix}
    C^{(1)} \\
    C^{(2)}
\end{pmatrix} = \begin{pmatrix}
h_l^{(1)}(kx_m) & h_l^{(2)}(kx_m) \\
h_l^{(1)'}(kx_m) & h_l^{(2)'}(kx_m)
\end{pmatrix}^{-1} \begin{pmatrix}
\chi(x_m) \\
\chi'(x_m)
\end{pmatrix}
\end{equation}